In [441]:
#############################################
#  p1: should we use softmax to get the loss?
#  ans: yes

#  p2: Then, log P(c|s) is computed as the average of log-probabilities of each composing token.
# There is a lot of "the" case

# max_candidate length

# Joan and Susan case?

#  p3: do not know how to slice the tensor in a easy way
# ans: masked tensor matmul
#############################################

from __future__ import absolute_import, division, print_function

from pytorch_transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForMaskedLM, BertTokenizer)

from pytorch_transformers import AdamW, WarmupLinearSchedule

import argparse
import glob
import logging
import os
import random

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

from torch.utils.data.distributed import DistributedSampler
from tensorboardX import SummaryWriter
from tqdm import tqdm, trange

import torch.nn.functional as F

logger = logging.getLogger(__name__)


In [442]:
# pop up example

# config = BertConfig.from_pretrained('bert-base-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM(config)
# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute. Your dog is not")).unsqueeze(0)  # Batch size 1

# # # masked = torch.tensor([-1, -1, -1, -1, 0, -1])

# outputs = model(input_ids, masked_lm_labels=input_ids)

# loss, prediction_scores = outputs[:2]

# # print(input_ids)
# # print(type(loss))
# print(prediction_scores)
# prob = F.softmax(prediction_scores[0], dim=0)
# print(prob)

# prob2 = F.softmax(prediction_scores, dim=1)
# print(prob2)

# print(type(prediction_scores.tolist()[0][4][2]))

# tokenizer.encode("Joan")
# print(tokenizer.convert_ids_to_tokens([7437]))
# seq = tokenizer.tokenize("dog is here")
# seq2 = tokenizer.convert_tokens_to_ids("dog is here")
# print(type(seq))

# mask = [-1] * len(seq)
# # mask[seq.index(2003)] = 1

# print(mask)




In [443]:
print(torch.tensor(0.0))

tensor(0.)


In [444]:
def get_loss(p1s, p2s, alpha, beta):
    # possible improvement if we use matrix computation
    m_p1 = np.mean(np.log(p1s)).item()
    m_p2 = np.mean(np.log(p2s)).item()
    loss = -1 * m_p1 + alpha * max(0, m_p2 - m_p1 + beta)
    
    return loss

In [445]:
# # t = torch.Tensor([1, 2, 3])
# # print ((t == 2).nonzero())

t = np.array([[6294, -1, -1,-1,  -1,   -1,   -1,   -1]])
print(np.where(t==-1))
t = np.delete(t, np.where(t == -1),axis=None)
print(t)

(array([0, 0, 0, 0, 0, 0, 0], dtype=int64), array([1, 2, 3, 4, 5, 6, 7], dtype=int64))
[]


In [446]:
# Config

class Args:
    
    data_dir = ".\examples\glue_data\WINO"
    model_name = 'bert-base-uncased'
    output_dir = ".\examples\glue_data\WNLI_out"
    task_name = "wino"
    output_mode = "classification"
    
    max_seq_length = 80
    max_cand_length = 8
    do_train = True
    do_eval = True
    evaluate_during_training = False
    do_lower_case = True
    
    per_gpu_train_batch_size = 4
    per_gpu_eval_batch_size = 4
    gradient_accumulation_steps = 1
    
    # for loss
    alpha = 5
    beta= 0.1
    
    learning_rate = 2e-5
    weight_decay = 0
    adam_epsilon = 1e-8
    max_grad_norm = 1.0
    num_train_epochs = 3
    max_steps = 1
    warmup_steps = 0
    
    logging_steps = 50
    save_steps = 50
    
    n_gpu = 1
    
    eval_all_checkpoints = False
    no_cuda = False
    overwrite_output_dir = True
    overwrite_cache = False

    fp16 = False
    fp16_opt_level = False
    local_rank = -1
    server_ip = ""
    server_port = ""

args=Args()

seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if args.n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

In [447]:
# Dataloader Utilization
import csv
import sys
from io import open

from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, label, first_token, second_token, pronoun, text_b=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: 0 means the first token is right,1 otherwise
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label
        self.first_candidate = first_token
        self.second_candidate = second_token
        self.pronoun = pronoun
     
    
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, masked_lm_labels, label_id, first_token_id, second_token_id):
        self.input_ids = input_ids
        self.masked_lm_labels = masked_lm_labels
        
        self.label_id = label_id
        self.first_token_id = first_token_id
        self.second_token_id = second_token_id
        
        
class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines
        

class WinoProcessor(DataProcessor):
    """Processor for the Wino."""
    # data type: tsv: guid - text - label - first cand - second cand

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text = line[1]
            # text_b = line[2]
            label = line[2]
            first_token = line[3]
            second_token = line[4]
            pronoun = line[5]
            
            examples.append(
                InputExample(guid,text, label, first_token, second_token, pronoun))
        return examples

    
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode,
                                 cls_token_at_end=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d" % (ex_index, len(examples)))

        tokens_a = tokenizer.tokenize(example.text_a)
        pronoun = example.pronoun

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = tokens_a + [sep_token]
        segment_ids = [sequence_a_segment_id] * len(tokens)
        
        if tokens_b:
            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

        if cls_token_at_end:
            tokens = tokens + [cls_token]
            segment_ids = segment_ids + [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            segment_ids = [cls_token_segment_id] + segment_ids
        
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        
        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
        else:
            input_ids = input_ids + ([pad_token] * padding_length)
            segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

        assert len(input_ids) == max_seq_length
        

        mask = [-1] * len(input_ids)
        mask[input_ids.index(tokenizer.encode(pronoun)[0])] = 0
        first_token_id = tokenizer.encode(example.first_candidate)
        second_token_id = tokenizer.encode(example.second_candidate)
        
        while len(first_token_id) < 8:
            first_token_id.append(-1)
        
        while len(second_token_id) < 8:
            second_token_id.append(-1)

        if output_mode == "classification":
            label_id = label_map[example.label]
        elif output_mode == "regression":
            label_id = float(example.label)
        else:
            raise KeyError(output_mode)

        features.append(
                InputFeatures(input_ids=input_ids,
                              masked_lm_labels=mask,
                              label_id=label_id,
                              first_token_id=first_token_id,
                              second_token_id=second_token_id))
        

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()    

            
def simple_accuracy(preds, labels):
    return (preds == labels).mean()


def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }


def pearson_and_spearman(preds, labels):
    pearson_corr = pearsonr(preds, labels)[0]
    spearman_corr = spearmanr(preds, labels)[0]
    return {
        "pearson": pearson_corr,
        "spearmanr": spearman_corr,
        "corr": (pearson_corr + spearman_corr) / 2,
    }


def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    
    if task_name == "wino":
        return {"acc": simple_accuracy(preds, labels)}


processors = {
    "wino": WinoProcessor
}

output_modes = {
    "wino": "classification"
}



In [448]:
# setting up envir

if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train and not args.overwrite_output_dir:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))

# Setup distant debugging if needed
if args.server_ip and args.server_port:
    # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
    import ptvsd
    print("Waiting for debugger attach")
    ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
    ptvsd.wait_for_attach()

# Setup CUDA, GPU & distributed training
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend='nccl')
    args.n_gpu = 1
args.device = device

# Setup logging
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO if args.local_rank in [-1, 0] else logging.WARN)
logger.warning("Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
                args.local_rank, device, args.n_gpu, bool(args.local_rank != -1), args.fp16)

# Load pretrained model and tokenizer
if args.local_rank not in [-1, 0]:
    torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab
    
    
num_labels = 2
config = BertConfig.from_pretrained(args.model_name)
tokenizer = BertTokenizer.from_pretrained(args.model_name)
model = BertForMaskedLM(config)

if args.local_rank == 0:
    torch.distributed.barrier()
    
model.to(args.device)
if args.local_rank != -1:
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank],
                                                      output_device=args.local_rank,
                                                      find_unused_parameters=True)
elif args.n_gpu > 1:
    model = torch.nn.DataParallel(model)

07/27/2019 04:39:23 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/27/2019 04:39:24 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at C:\Users\xzhao\.cache\torch\pytorch_transformers\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
07/27/2019 04:39:24 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "tor

In [449]:
def train(args, train_dataset, model, tokenizer):
    
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args.warmup_steps, t_total=t_total)
    
    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)
        
    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
                   args.train_batch_size * args.gradient_accumulation_steps * (torch.distributed.get_world_size() if args.local_rank != -1 else 1))
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)
    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0])
    # set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
#             inputs = {'input_ids':      batch[0],
#                       'attention_mask': batch[1],
#                       'token_type_ids': batch[2],  
#                       'labels':         batch[3]}
            
            inputs = {'input_ids':      batch[0],
                      'masked_lm_labels': batch[1]}
            
            input_ids = batch[0]
            mask_lm_labels = batch[1]
            
            correct_token_labels = batch[2]
            
            # Assume that there is only one token for pronoun
           
            batch_first_token_ids = batch[3].cpu().detach().numpy()
            batch_second_token_ids = batch[4].cpu().detach().numpy()

            
            ouputs = model(**inputs)
            scores = F.softmax(ouputs[1], dim=1)
            scores = scores.cpu().detach().numpy()

            
            loss = torch.tensor(0.0)
            
            for i in range(len(batch[0])):
                batch_real_first = np.delete(batch_first_token_ids[i], np.where(batch_first_token_ids[i] == -1))
                batch_real_second = np.delete(batch_second_token_ids[i], np.where(batch_second_token_ids[i] == -1))
                pronoun_index = (mask_lm_labels[i] == 0).nonzero()
                
                if correct_token_labels[i] == 0:
                    true_probs = scores[i][pronoun_index][batch_real_first]
                    wrong_probs = scores[i][pronoun_index][batch_real_first]
                elif correct_token_labels[i] == 1:
                    wrong_probs = scores[i][pronoun_index][batch_real_second]
                    true_probs = scores[i][pronoun_index][batch_real_second]
                
                loss += get_loss(true_probs, wrong_probs, args.alpha, args.beta)
            
            loss = loss / len(batch[0])
            loss = torch.autograd.Variable(loss, requires_grad = True)
            
            if args.n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                scheduler.step()  # Update learning rate schedule
                optimizer.step()
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if args.local_rank == -1 and args.evaluate_during_training:  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar('eval_{}'.format(key), value, global_step)
                    tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar('loss', (tr_loss - logging_loss)/args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    torch.save(args, os.path.join(output_dir, 'training_args.bin'))
                    logger.info("Saving model checkpoint to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

In [450]:
def evaluate(args, model, tokenizer, prefix=""):
    
    eval_task_names = [args.task_name]
    eval_outputs_dirs = [args.output_dir]
                         
    results = {}
    for eval_task, eval_output_dir in zip(eval_task_names, eval_outputs_dirs):
        eval_dataset = load_and_cache_examples(args, eval_task, tokenizer, evaluate=True)

        if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(eval_output_dir)

        args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
        # Note that DistributedSampler samples randomly
        eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

        # Eval!
        logger.info("***** Running evaluation {} *****".format(prefix))
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(args.device) for t in batch)

            with torch.no_grad():
                inputs = {'input_ids':      batch[0],
                      'masked_lm_labels': batch[1]}

                
                input_ids = batch[0]
                mask_lm_labels = batch[1]
                
                tmp_eval_loss = torch.tensor(0.0)
            
                correct_token_labels = batch[2]
            
                # Assume that there is only one token for pronoun
                batch_first_token_ids = batch[3].cpu().detach().numpy()
                batch_second_token_ids = batch[4].cpu().detach().numpy()

                ouputs = model(**inputs)
                scores = F.softmax(ouputs[1], dim=1)
                scores = scores.cpu().detach().numpy()

                for i in range(len(batch[0])):
                    
                    batch_real_first = np.delete(batch_first_token_ids[i], np.where(batch_first_token_ids[i] == -1))
                    batch_real_second = np.delete(batch_second_token_ids[i], np.where(batch_second_token_ids[i] == -1))
                    
                    pronoun_index = (mask_lm_labels[i] == 0).nonzero()
                    
                    p1 = scores[i][pronoun_index][batch_real_first]
                    p2 = scores[i][pronoun_index][batch_real_second]
                    
                    if correct_token_labels[i] == 0:
                        true_probs = p1
                        wrong_probs = p2
                    elif correct_token_labels[i] == 1:
                        wrong_probs = p1
                        true_probs = p2

                    tmp_eval_loss += get_loss(true_probs, wrong_probs, args.alpha, args.beta)
                    
                    if preds is None:
                        mp1 = np.mean(np.log(p1)).item()
                        mp2 = np.mean(np.log(p2)).item()
                        
                        preds = bool(mp1 > mp2)
                        
                    else:
                        mp1 = np.mean(np.log(p1)).item()
                        mp2 = np.mean(np.log(p2)).item()
                        preds = np.append(preds, bool(mp1 > mp2))
                        
                if out_label_ids is None:
                    out_label_ids = batch[2].detach().cpu().numpy()
                else:
                    out_label_ids = np.append(out_label_ids, batch[2].detach().cpu().numpy(), axis=0)
                    
                tmp_eval_loss = tmp_eval_loss / len(batch[0])                
                
                eval_loss += tmp_eval_loss.mean().item()
            
            
            nb_eval_steps += 1
        eval_loss = eval_loss / nb_eval_steps
        
#         if args.output_mode == "classification":
#             preds = np.argmax(preds, axis=1)
#         elif args.output_mode == "regression":
#             preds = np.squeeze(preds)
        result = compute_metrics(eval_task, preds, out_label_ids)
        results.update(result)

        output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results {} *****".format(prefix))
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

    return results

In [451]:
def load_and_cache_examples(args, task, tokenizer, evaluate=False):
    
    processor = processors[task]()
    output_mode = output_modes[task]
    
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(args.data_dir, 'cached_{}_{}_{}_{}'.format(
        'dev' if evaluate else 'train',
        list(filter(None, args.model_name)).pop(),
        str(args.max_seq_length),
        str(task)))
    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args.data_dir) if evaluate else processor.get_train_examples(args.data_dir)
        features = convert_examples_to_features(examples, label_list, args.max_seq_length, tokenizer, output_mode,
            cls_token_at_end = False,            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id = 1,
            pad_on_left= 0,                 # pad on the left for xlnet
            pad_token_segment_id = 0
                )                                    
        if args.local_rank == [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)

                       

    # Convert to Tensors and build dataset
    
#     print(features[0].input_ids)
#     print(features[0].masked_lm_labels)
#     print(features[0].first_token_id)
    
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_masked_lm_labels = torch.tensor([f.masked_lm_labels for f in features], dtype=torch.long)
    all_first_token_id = torch.tensor([f.first_token_id for f in features], dtype=torch.long)
    all_second_token_id = torch.tensor([f.second_token_id for f in features], dtype=torch.long)
    
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_masked_lm_labels, all_label_ids, all_first_token_id,all_second_token_id)
    return dataset

In [452]:
# Do training

# Training
if args.do_train:
    train_dataset = load_and_cache_examples(args, args.task_name, tokenizer, evaluate=False)
    global_step, tr_loss = train(args, train_dataset, model, tokenizer)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)


# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
    # Create output directory if needed
    if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
        os.makedirs(args.output_dir)

    logger.info("Saving model checkpoint to %s", args.output_dir)
    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)

    # Good practice: save your training arguments together with the trained model
    torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

    # Load a trained model and vocabulary that you have fine-tuned

    model = BertForMaskedLM.from_pretrained(args.output_dir)
    tokenizer = BertTokenizer.from_pretrained(args.output_dir)
    model.to(args.device)


# Evaluation
results = {}
if args.do_eval and args.local_rank in [-1, 0]:
    checkpoints = [args.output_dir]
    if args.eval_all_checkpoints:
        checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + '/**/' + WEIGHTS_NAME, recursive=True)))
        logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
        model = BertForMaskedLM.from_pretrained(args.output_dir)
        model.to(args.device)
        result = evaluate(args, model, tokenizer, prefix=global_step)
        result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
        results.update(result)

print(results)

07/27/2019 04:39:27 - INFO - __main__ -   Creating features from dataset file at .\examples\glue_data\WINO
07/27/2019 04:39:27 - INFO - __main__ -   Writing example 0 of 279
07/27/2019 04:39:28 - INFO - __main__ -   ***** Running training *****
07/27/2019 04:39:28 - INFO - __main__ -     Num examples = 279
07/27/2019 04:39:28 - INFO - __main__ -     Num Epochs = 1
07/27/2019 04:39:28 - INFO - __main__ -     Instantaneous batch size per GPU = 4
07/27/2019 04:39:28 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 4
07/27/2019 04:39:28 - INFO - __main__ -     Gradient Accumulation steps = 1
07/27/2019 04:39:28 - INFO - __main__ -     Total optimization steps = 1































Epoch:   0%|                                                                                  | 0/1 [00:00<?, ?it/s]































Iteration:   0%|                                                                             | 0/70 [00:00<?, ?

Evaluating:  71%|███████████████████████████████████████████████▊                   | 50/70 [00:03<00:01, 15.43it/s]






























Evaluating:  74%|█████████████████████████████████████████████████▊                 | 52/70 [00:03<00:01, 15.43it/s]






























Evaluating:  77%|███████████████████████████████████████████████████▋               | 54/70 [00:03<00:01, 15.43it/s]






























Evaluating:  80%|█████████████████████████████████████████████████████▌             | 56/70 [00:03<00:00, 15.32it/s]






























Evaluating:  83%|███████████████████████████████████████████████████████▌           | 58/70 [00:03<00:00, 15.42it/s]






























Evaluating:  86%|█████████████████████████████████████████████████████████▍         | 60/70 [00:03<00:00, 15.46it/s]






























Evaluating:  89%|███████████████████████████████████████████████████████████▎       | 62/70 [00:04<00:00, 15.49it/s]



[False False  True  True False False False False  True  True  True  True
 False  True  True  True  True  True False False False False False  True
  True  True False False  True  True  True  True  True  True  True  True
  True  True  True  True False False False False  True  True  True  True
  True  True  True  True  True  True  True  True False False False False
  True  True False False False False  True  True False False False False
  True  True  True  True False False False False False False False False
 False False  True  True  True  True False False  True  True  True  True
  True  True  True  True  True  True False False False False False False
  True  True False False  True  True  True  True  True False  True  True
  True  True False False  True  True False False False False False False
 False False  True  True False False  True  True  True  True  True  True
 False False  True  True False False  True  True False False  True  True
  True  True  True  True  True  True  True  True  T

07/27/2019 04:39:40 - INFO - __main__ -   ***** Eval results  *****
07/27/2019 04:39:40 - INFO - __main__ -     acc = 0.5053763440860215


{'acc_': 0.5053763440860215}
